<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura: Data preprocessing</center>

# Challenge S2: Anonimización AirBnbMadrid

## Objetivos

En este challenge nos enfrentaremos a un dataset real, que contiene los datos de los alojamientos disponibles de Airbnb para la comunidad de Madrid. Nuestro objetivo en el challenge es anonimizarlo y convertir el dataset de AirBnBMadrid a AirBnBValladolid.

## Configuración del entorno

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from faker import Faker
from os import remove
from os import path
import re

fake = Faker('es_ES')

## Carga de los datos
Esta vez va a ser fácil, vamos a importar los datos de un fichero csv, utilizaremos la función read_csv que nos proporciona la libreria de pandas.

In [2]:
df = pd.read_csv('dataset_airbnb_madrid.csv')

Como este dataset es muy complejo, vamos a quedarnos con un subconjunto de columnas para este challenge:

In [3]:
df = df[['id', 'listing_url', 'name', 'summary', 'price', 'weekly_price', 'zipcode', 'country', 'latitude', 'longitude']]

## Comprensión del dataset

Una vez cargados los datos debemos inspeccionarlos, y entender que datos contiene cada una de las columnas:

Describe por cada columna:

* ¿Qué contiene?
* ¿Cual es el rango de los datos?
* ¿Contiene datos sensibles?
* ¿Depende de otras?

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13335 entries, 0 to 13334
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            13335 non-null  int64  
 1   listing_url   13335 non-null  object 
 2   name          13335 non-null  object 
 3   summary       12846 non-null  object 
 4   price         13335 non-null  object 
 5   weekly_price  3512 non-null   object 
 6   zipcode       12896 non-null  object 
 7   country       13334 non-null  object 
 8   latitude      13335 non-null  float64
 9   longitude     13335 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 1.0+ MB


In [5]:
df.head()

,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,7830063,https://www.airbnb.com/rooms/7830063,Quiet room in Plaza Mayor,Room in magnificent property in the historic c...,$42.00,$300.00,28005,Spain,40.412275,-3.708718
1,9898596,https://www.airbnb.com/rooms/9898596,Homely apartment in the heart of Madrid,"Spacious apartment for up to 10 people, with a...",$135.00,NaN,28005,Spain,40.411093,-3.708985
2,15334645,https://www.airbnb.com/rooms/15334645,Piso Muy Luminoso en pleno centro de Madrid,"Lugares de interés: Casa Lucio, Cine Doré, Cal...",$81.00,NaN,28005,Spain,40.413587,-3.708945
3,1307795,https://www.airbnb.com/rooms/1307795,Rent room in the heart of Madrid,NaN,$43.00,$240.00,28013,Spain,40.419936,-3.709180
4,17410608,https://www.airbnb.com/rooms/17410608,Luxury duplex penthouse in historic building,Amazing duplex penthouse in a historic buildin...,$50.00,NaN,28005,Spain,40.410894,-3.712537


### id

¿Que contiene? - Contiene el identificador del anuncio. Es del tipo int64.

¿Cual es el rango de los datos?

In [6]:
print('El rango de la columna id va del mínimo : ',df['id'].min() , ' al máximo : ', df['id'].max())

El rango de la columna id va del mínimo :  18628  al máximo :  18109842


¿Contiene datos sensibles? - Si. Aunque no son datos personales o de salud, si podrían detectar el anuncio real

¿Depende de otras? - Si. listin_url contiene la información de id.

### listing_url

¿Que contiene? - Contiene la url del anuncio. Es del tipo object ya que es una cadena de caracteres.

¿Cual es el rango de los datos?

In [7]:
#Como la columna listing_url tiene una clara dependencia de id el criterio para calcular el máximo y el mínimo 
#será el que tenga el id maximo y mínimo en la url.

min=df['listing_url'].loc[df['listing_url'].str.contains(str(df['id'].min()))]
max=df['listing_url'].loc[df['listing_url'].str.contains(str(df['id'].max()))]

print('El rango de la columna listing_url va del mínimo : ',min, ' al máximo : ', max)

El rango de la columna listing_url va del mínimo :  4074    https://www.airbnb.com/rooms/18628
Name: listing_url, dtype: object  al máximo :  4994    https://www.airbnb.com/rooms/18109842
Name: listing_url, dtype: object


¿Contiene datos sensibles?
Si. Aunque no son datos personales o de salud, si podrían detectar el anuncio real

¿Depende de otras?
Si del id del anuncio que forma parte de la url

### name

¿Que contiene? - Es el nombre del anuncio. Es un object, ya que contiene cadenas de caracteres.

¿Cual es el rango de los datos? - Al ser un texto libre el rango que podemos calcular es de longitud del texto.

In [8]:
index_min=df.loc[:,'name'].str.len().idxmin()
min=df.loc[:,'name'].str.len().min()
index_max=df.loc[:,'name'].str.len().idxmax()
max=df.loc[:,'name'].str.len().max()
print('El rango de la columna name va del mínimo : ',min, ' en la fila ', index_min, ' al máximo : ', max, ' en la fila ', index_max)

El rango de la columna name va del mínimo :  1  en la fila  983  al máximo :  75  en la fila  10906


¿Contiene datos sensibles? No. Es un texto libre. Puede aparecer alguna vez la palabra Madrid

¿Depende de otras? No. 

### summary

¿Que contiene? - Es el cuerpo o descripción del anuncio. Es un object, ya que contiene cadenas de caracteres.

¿Cual es el rango de los datos? - Al ser un texto libre el rango no es medible

In [9]:
index_min=df.loc[:,'summary'].str.len().idxmin()
min=df.loc[:,'summary'].str.len().min()
index_max=df.loc[:,'summary'].str.len().idxmax()
max=df.loc[:,'summary'].str.len().max()
print('El rango de la columna name va del mínimo : ',min, ' en la fila ', index_min, ' al máximo : ', max, ' en la fila ', index_max)

El rango de la columna name va del mínimo :  1.0  en la fila  3596  al máximo :  1000.0  en la fila  380


¿Contiene datos sensibles? No. En principio describe al propietario y el alojaomiento. Puede aparecer alguna vez la palabra Madrid

¿Depende de otras? Si. Si en name aparece la mención a Madrid habría que ver si existe aquí y sustituirlo.

### price

¿Que contiene? - Es el precio por noche del alojamiento. Es un object. El precio viene representado por una cadena de caracteres.

¿Cual es el rango de los datos?

In [10]:
# Esta función tranforma los importes que vienen como object en floats.
# No tenemos en cuenta el primer caracter que es la divisa y sustituimos la "," de los miles por ""

def transformar_importes(importe):
    if type(importe) == str:
        temp = float(importe[1:].replace(',', ''))
    else:
        temp = np.nan
    return temp

In [11]:
#Creamos una columna copia ya que para hacer los calculos no queremos cambiar la columna orignal del dataframe original
df_copy=df['price']

#Aplicamos mediante apply la transformación en float mediante la función transformar_importes

df_copy=df_copy.apply(transformar_importes)
print('El rango de la columna price va del mínimo : ',df_copy.min(), ' al máximo : ', df_copy.max())

El rango de la columna price va del mínimo :  9.0  al máximo :  7700.0


¿Contiene datos sensibles?- No. Son el importe del precio de estancia por un día.

¿Depende de otras? No.

### weekly_price

¿Que contiene? - Es el precio por semana del alojamiento. Es un object. El precio viene representado por una cadena de caracteres.

¿Cual es el rango de los datos?

In [12]:
df_copy['weekly_price']=df['weekly_price']
df_copy['weekly_price']=df_copy['weekly_price'].apply(transformar_importes)
print('El rango de la columna weekly_price va del mínimo : ',df_copy['weekly_price'].min(), ' al máximo : ', df_copy['weekly_price'].max())

El rango de la columna weekly_price va del mínimo :  70.0  al máximo :  7000.0


¿Contiene datos sensibles? No. Son el importe del alquiler por semana.

¿Depende de otras? No.

### zipcode

¿Que contiene? - Es el código postal donde se situa el inmueble a alquilar. Es un object.

¿Cual es el rango de los datos?

In [13]:
def transformar_codigos_postales(cp):
    if type(cp)==str:
        cp=cp.strip().replace("-","") # Hacemos varios ajustes para alinear cp y se puedan transformar en int
        if cp:
            temp = int(cp[-5:])
        else:
            temp=28001 # Cadenas vacias las ponemos el primer código postal de Madrid
    else:
        temp=28001 # na las ponemos el primer código postal de Madrid
        
    return temp

In [14]:
df['zipcode']=df['zipcode'].apply(transformar_codigos_postales)
print('El rango de la columna zipcode va del mínimo : ',df['zipcode'].min(), ' al máximo : ', df['zipcode'].max())

El rango de la columna zipcode va del mínimo :  28  al máximo :  84084


¿Contiene datos sensibles? No, es el código postal del barrio donde se situa el inmueble.

¿Depende de otras? Si, parcialmente si el name o summary aparece la ciudad de Madrid habría que cambiarlo.

### Country

¿Que contiene? - El país del inmueble. Es del tipo Object, por que el pais se representa por su ombre completo en una cadena de caracteres

¿Cual es el rango de los datos?

In [15]:
df['country'].value_counts()

Spain    13333
Cuba         1
Name: country, dtype: int64

¿Contiene datos sensibles? No. Es el país de la ciudad donde se situa el inmueble

¿Depende de otras? No. En este caso. Si las ciudades fueran de de distintos paises si habría dependencia.

### latitude y longitude

¿Que contiene? - La latitud y longitud geográfica donde se encuentra el inmueble que se alquila.

¿Cual es el rango de los datos?

In [16]:
print('El rango de la columna latitude va del mínimo : ',df['latitude'].min(), ' al máximo : ', df['latitude'].max())

El rango de la columna latitude va del mínimo :  40.33188827449475  al máximo :  40.562736291267576


In [17]:
print('El rango de la columna latitude va del mínimo : ',df['longitude'].min(), ' al máximo : ', df['longitude'].max())

El rango de la columna latitude va del mínimo :  -3.863907266766836  al máximo :  -3.5268214016469366


¿Contiene datos sensibles? No.

¿Depende de otras? No. La dependencia podría tenerla de la columna ciudad.

## Dependencias entre variables
A continuación os propongo hacer una matriz de dependencias para analizar que variables dependen entre sí y analizar que grupos de variables existen. Esto es importante de cara a la anonimización del dataset, puesto que dejar una de estas variables dependientes sin anonimizar, podría llevar a "deshacer" este proceso a personas malintencionadas. 

En esta matriz podéis marcar con una _X_ qué variables dependen entre sí. Si creéis que existen varios grupos de dependencia, asignar una marca diferente a cada uno de los grupos que consideréis. Por ejemplo, variables que dependen entre sí, podrían ser la dirección de la calle y el código postal.

| Depende de   | id | listing_url | name | summary | price | weekly_price | zipcode | country | lat y long |
|---           |--- |---          |---   |---      |---    |---           |---      |---      |---         | 
| **id**           | x   |    x         |      |         |       |              |         |         |            | 
| **listing_url**  | x   |   x          |      |         |       |              |         |         |            | 
| **name**         |    |             |   x   |   x      |       |             |         |         |            | 
| **summary**      |    |             |   x   |   x     |       |             |         |         |            | 
| **price**        |    |            |      |         |    x   |              |         |         |            | 
| **weekly_price** |    |             |      |        |       |       x       |         |         |            | 
| **zipcode**      |    |             |      |         |       |             |      x   |         |            | 
| **country**      |    |             |      |         |       |              |         |    x     |            | 
| **lat y long**   |    |             |      |         |       |              |         |         |     x       | 



## Estrategia de anonimización

A partir de aquí por cada grupo de variables dependientes determina cual es la estrategia de anonimización más adecuada y aplícala teniendo en cuenta las dependencias encontradas en el paso anterior.

#### Estrategia de anonimización

* Vamos anonimizar las columnas id y listing_url. Si cambiamos el id lo tenemos que cambiar en la url contenida en listing_url.

* Vamos anonimizar name y summary, para sustituir la palabra Madrid, si aparece, por Valladolid

* Vamos anonimizar zipcode poniendo el cp de Valladolid 47001

* Vamos anonimizar lat y long por las de Valladolid latidud = 41,65518 y longitud =	-4,72372

* Vamos anonimizar el país por Spain que es donde se situa Valladolid. (Lo hacemos por que se ve que hay un Cuba)



In [18]:
def anon_Madrid_x_Valladolid(row):
    # Anonimizamos el id del anuncio
    id_anuncio= fake.numerify(text="########")
    row['id'] = id_anuncio
    row['listing_url'] =  'https://www.airbnb.com/rooms/'+id_anuncio
    
    #Sustituimos las apariones de Madrid x Valladolid.
    #Utilizamos re regular expresión para que nos detecte tanto en mayúsculas como en minúsculas
    redata = re.compile(re.escape('madrid'), re.IGNORECASE)
    row['name']=redata.sub('Valladolid', row['name'])
    row['summary']=redata.sub('Valladolid', str(row['summary']))
    
    #Sustituimos el código postal por el de Valladolid
    row['zipcode'] = 47001
    
    #Sustituimos la latitud y longitud las de Valladolid
    
    row['latitude'] = float(41.65518)
    row['longitude'] = float(-4.72372)
    
    #Sustituimos el país por Spain que es dode se situa Valladolid
    
    row['country']='Spain'
    
    
    return row

In [19]:
#Aplicamos la anonimización utilizando apply y la función anon_Madrid_x_Valladolid. Se ejecuta por fila axis=1
df = df.apply(anon_Madrid_x_Valladolid, axis=1)
df.head()

,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,25368493,https://www.airbnb.com/rooms/25368493,Quiet room in Plaza Mayor,Room in magnificent property in the historic c...,$42.00,$300.00,47001,Spain,41.65518,-4.72372
1,37783098,https://www.airbnb.com/rooms/37783098,Homely apartment in the heart of Valladolid,"Spacious apartment for up to 10 people, with a...",$135.00,NaN,47001,Spain,41.65518,-4.72372
2,89023336,https://www.airbnb.com/rooms/89023336,Piso Muy Luminoso en pleno centro de Valladolid,"Lugares de interés: Casa Lucio, Cine Doré, Cal...",$81.00,NaN,47001,Spain,41.65518,-4.72372
3,72569025,https://www.airbnb.com/rooms/72569025,Rent room in the heart of Valladolid,nan,$43.00,$240.00,47001,Spain,41.65518,-4.72372
4,01278031,https://www.airbnb.com/rooms/01278031,Luxury duplex penthouse in historic building,Amazing duplex penthouse in a historic buildin...,$50.00,NaN,47001,Spain,41.65518,-4.72372


In [20]:
# Esta función escribe un fichero en formato csv a partir de un dataframe dado

def escribir_csv (df,nombre_fich):

    if path.exists(nombre_fich): # Si existe ya el fichero lo borramos para volver a generarlo
        remove(nombre_fich)
        
    df.to_csv(nombre_fich)
    
    return

In [21]:
# Generamos el nuevo csv de Valladolid
escribir_csv(df,'dataset_airbnb_valladolid.csv')

## Conclusiones

* ¿Qué ventajas / inconvenientes le ves a esta manera de anonimizar?
<br>

    - *Ventajas: Que es una forma sencilla y rápida de realizarla, para todas aquellas columnas que sus datos son númericos o categóricos*.
    <br>
    
    - *Inconvenientes:* 
        - *En columnas en los que se les permite meter lenguaje natural, como name, summary, incluso en menor medida zipcode, es muy dificil anonimizar ya que hay que detectar todas las posibles palabras o frases enteras que pudieran identificar este caso Madrid y sustituirlos por expresiones equivalentes que pudieran ser representativas de Valladolid.*
        - *Así también complica la tarea de que estos campos están redactados en distintos idiomas, en este ejemplo que haya podido detectar el summary y el name están redactados en español, ingles, en algún idioma Oriental (Chino, Japones ...)*
        - *Otro tema más entiendo que más complejo es si ya tuvieramos que anonimizar imagenes del inmueble y fachada y demás ...*
    
   
* ¿Cómo podríamos mejorar los algoritmos?
<br>

    - *Haciendo más fuerte la relación de todos los cambios de anonimización, de tal forma que si anonimiza un datoel restode datos sensibles también se haga de la misma forma, así como si se quieren revertir estos cambios todos se hagan de la misma forma y relacionados*
    <br>

    - *Principalmente por lo que he podido ver, sería la generación de un "diccionario" tanto de Barrios, Lugares descripciones, y sus equivalencias con otras ciudades de cara a poder crear un algoritmo que las detecte y haga la sustitución y que además se tenga en cuenta tanto para el "diccionario" como para el algoritmo que los textos están redactados en multiples idiomas.*
     <br>

    - *Aunque en este ejemplo no se da sería el tema de las imagenes, el crear un banco de imagenes que te las pudiera cambiar dependiendo de lo que se describe en el name o en el summary*
   
    <br>
    
* ¿Cómo podríamos evaluar que una anonimización es buena?
<br>

    - *Se me ocurre que alguién de Valladolid al consultar el anuncio lo percibiera como un anuncio real de la ciudad o por el contrario que alguien de Madrid no reconociera el anuncio como un anuncio de Madrid y le fuera todo ajeno.*